# Sample 2: Cloud Build

Reference: cloud.google.com/build/docs/overview

## Build and Push a Docker image with Cloud Build
Reference: https://cloud.google.com/build/docs/build-push-docker-image

In [ ]:
# Enable APIs
# Cloud Build and Artifact Registry APIs

In [ ]:
# gcloud init

### Prepare source files to build

In [ ]:
%%sh
mkdir quickstart-docker

In [ ]:
cd quickstart-docker

In [ ]:
%%writefile quickstart.sh
echo "Hi"

In [ ]:
%%writefile Dockerfile
FROM ubuntu
COPY quickstart.sh /
CMD ["/quickstart.sh"]

In [ ]:
!chmod +x quickstart.sh

### Create a Docker repository in Artifact Registry

In [ ]:
%%sh
# Create a Docker repository in Artifact Registry
gcloud artifacts repositories create quickstart-docker-repo \
    --repository-format=docker \
    --location=us-west2 \
    --description="Docker repository"

In [ ]:
%%sh
# Verify
gcloud artifacts repositories list

### Build an image using Dockerfile

In [ ]:
!gcloud config get-value project

In [ ]:
!gcloud builds submit \
    --region=us-west2 \
    --tag us-west2-docker.pkg.dev/$(gcloud config get-value project)/quickstart-docker-repo/quickstart-image:tag1

### Build an image using a build config file

In [ ]:
%%writefile cloudbuild.yaml
steps:
- name: 'gcr.io/cloud-builders/docker'
  script: |
    docker build -t us-west2-docker.pkg.dev/$PROJECT_ID/quickstart-docker-repo/quickstart-image:tag1 .
  automapSubstitutions: true
images:
- 'us-west2-docker.pkg.dev/$PROJECT_ID/quickstart-docker-repo/quickstart-image:tag1'

In [ ]:
!gcloud builds submit \
    --region=us-west2 \
    --config cloudbuild.yaml

### Clean up

In [ ]:
# Delete quickstart-docker-repo

In [ ]:
cd ..

## Deploy a containerized application to Cloud Run using Cloud Build

Reference: https://cloud.google.com/build/docs/deploy-containerized-application-cloud-run

In [ ]:
# Enable APIs
# Cloud Build, Cloud Run, Artifact Registry, and Compute Engine APIs

In [ ]:
# gcloud init

### Grant Permissions

In [ ]:
!PROJECT_ID=$(gcloud config list --format='value(core.project)')
# Enable Cloud Resource Manager
!PROJECT_NUMBER=$(gcloud projects describe $PROJECT_ID --format='value(projectNumber)')

In [ ]:
# Grant Cloud Run Admin role to the CLoud Build service account
!gcloud projects add-iam-policy-binding $(gcloud config list --format='value(core.project)') \
    --member=serviceAccount:$(gcloud projects describe $PROJECT_ID --format='value(projectNumber)')@cloudbuild.gserviceaccount.com \
    --role=roles/run.admin

In [ ]:
# Grant the IAM Service Account User role to the Cloud Build service account for the Cloud Run runtime service account
!gcloud iam service-accounts add-iam-policy-binding \
    $(gcloud projects describe $PROJECT_ID --format='value(projectNumber)')-compute@developer.gserviceaccount.com \
    --member=serviceAccount:$(gcloud projects describe $PROJECT_ID --format='value(projectNumber)')@cloudbuild.gserviceaccount.com \
    --role=roles/iam.serviceAccountUser

### Deploy a prebuilt image

In [ ]:
!mkdir cloud-build-hello

In [ ]:
cd cloud-build-hello

In [ ]:
%%writefile cloudbuild.yaml
steps:
- name: 'gcr.io/cloud-builders/gcloud'
  script: |
    gcloud run deploy cloudrunservice --image us-docker.pkg.dev/cloudrun/container/hello --region us-central1 --platform managed --allow-unauthenticated

In [ ]:
!gcloud builds submit \
    --region=us-west2 \
    --config cloudbuild.yaml

### Clean up

In [ ]:
# Delete Cloud Run service

In [ ]:
cd ..

## Automate Builds

### Create a repository

In [ ]:
# I'll be using GCP Cloud Repositories.
# Create a repo and clone
!gcloud source repos clone cloud-build-automate-builds --project=training1027

In [ ]:
cd cloud-build-automate-builds

In [ ]:
!touch sample.txt

In [ ]:
!git add .
!git commit -m "Add and modify files"

In [ ]:
!git push -u origin master
# or 
# !git push -u origin main

### Create files

In [ ]:
%%writefile main.py
print("Success!")

In [ ]:
%%writefile cloudbuild.yaml
steps:
    - name: python
      script: |
        python3 main.py

### Build Triggers

In [ ]:
# Go to Cloud Build triggers and create one trigger 

In [ ]:
%%sh
# Commit the file to the repo
git add .
git commit -m "Add and modify files"
git push -u origin master

In [ ]:
# Success!

### Clean up

In [ ]:
# Delete resources

In [ ]:
cd ..

## Automate Builds and Deploy using Cloud Repositories, Cloud Build, and Cloud Run

In [ ]:
# Use Automate Builds repos

In [ ]:
cd cloud-build-automate-builds

In [ ]:
%%writefile cloudbuild.yaml
steps:
- name: 'gcr.io/cloud-builders/gcloud'
  script: |
    gcloud run deploy cloudrunservice --image us-docker.pkg.dev/cloudrun/container/hello --region us-central1 --platform managed --allow-unauthenticated

In [ ]:
%%sh
# Commit the file to the repo
git add .
git commit -m "Add and modify files"
git push -u origin master

In [ ]:
# Success!

### Clean up

In [ ]:
# Delete resources

In [ ]:
cd ..